# 2D Slab Benchmark

IMPORTANT: This code only works for finite toroidal angle. A finite poloidal angle has not been implemented yet

The set-up of the 2D Linear Layer: $n_e$ varies linearly with $X$, $\bold{B} = -1 \bold{\hat{z}}$, and the wave propagation is confined to the XY-plane. See: https://arxiv.org/pdf/2408.12919

In [ ]:
# constructing the data needed -- magnetic field data, ne data, polflux data

import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib
import scipy.constants as const

X = np.linspace(start = -0.1, stop = 1,   num = 221)  # original is num = 221
Y = np.linspace(start = -0.1, stop = 2,   num = 421)  # original is num = 421
Z = np.linspace(start = -0.5, stop = 0.5, num = 51)   # original is start = -0.2, end = 0.2, num = 41
Bx = np.zeros((len(X), len(Y), len(Z)))
By = np.zeros((len(X), len(Y), len(Z)))
Bz = np.full((len(X), len(Y), len(Z)), -1)

pol_flux_1D = np.linspace(start = 1.1, stop = 0, num = len(X))
pol_flux_3D = np.broadcast_to(np.expand_dims(pol_flux_1D, axis=(1,2)), (len(X), len(Y), len(Z)))
ne = np.linspace(start = -0.1, stop = 1, num = len(pol_flux_1D))
ne[pol_flux_1D >= 1.0] = 0

topfile_data = {
    "X": X.tolist(),
    "Y": Y.tolist(),
    "Z": Z.tolist(),
    "Bx": Bx.tolist(),
    "By": By.tolist(),
    "Bz": Bz.tolist(),
    "pol_flux": pol_flux_3D.tolist()
}

where_to_dump_topfile = r"C:\Users\eduar\OneDrive\OneDrive - EWIKARTA001\OneDrive - Nanyang Technological University\University\2. To Be Uploaded\Year 3, PH4416 Professional Attachment\Scotty in 3d Cartesian"

with open(os.path.join(where_to_dump_topfile, "topfiletest.json"), "w") as topfile:
    json.dump(topfile_data, topfile, indent=4)

ne_entries = len(pol_flux_1D)
ne_data = np.column_stack((pol_flux_1D, ne))



# The plots below are just to illustrate the set-up of the 2D linear layer
# The ne data is not actually generated because Scotty has a built-in
# function that does a linear fit for ne directly, so we use that instead

# plot of ne vs polflux
# Create the plot
plt.figure(figsize=(4, 3))
plt.plot(pol_flux_1D, ne, marker='o', linestyle='-', color='b')
# Add labels and title
plt.xlabel('pol_flux_1D')
plt.ylabel('ne')
plt.title('Plot of pol_flux_1D vs. ne')
plt.grid()
# Show the plot
plt.show()

# plot of polflux vs X
# Create the plot
plt.figure(figsize=(4, 3))
plt.plot(X, pol_flux_1D, marker='o', linestyle='-', color='r')
# Add labels and title
plt.xlabel('X')
plt.ylabel('pol_flux_1D')
plt.title('Plot of X vs. pol_flux_1D')
plt.grid()
# Show the plot
plt.show()

# plot of ne vs X
# Create the plot
plt.figure(figsize=(4, 3))
plt.plot(X, ne, marker='o', linestyle='-', color='g')
# Add labels and title
plt.xlabel('X')
plt.ylabel('ne')
plt.title('Plot of X vs. ne')
plt.grid()
# Show the plot
plt.show()

In [ ]:
import numpy as np
import pathlib
from scipy import constants
from scotty.beam_me_up_3D_temp import beam_me_up_3D
from scotty.fun_general import freq_GHz_to_wavenumber

kwargs_dict = {}
kwargs_dict["poloidal_launch_angle_Torbeam"] = 0.0
kwargs_dict["toroidal_launch_angle_Torbeam"] = 30.0
kwargs_dict["launch_position_cartesian"] = np.array([1e-5, 1e-5, 0.0])
kwargs_dict["launch_beam_width"] = 0.05
kwargs_dict["launch_beam_curvature"] = 0
kwargs_dict["launch_freq_GHz"] = 15.0
kwargs_dict["mode_flag"] = -1                                          # -1 for O-mode for my case

kwargs_dict["vacuumLaunch_flag"] = False
kwargs_dict["relativistic_flag"] = False
kwargs_dict["find_B_method"] = "omfit_3D"
kwargs_dict["density_fit_parameters"] = [1.0, 1.0]
kwargs_dict["temperature_fit_parameters"] = None
kwargs_dict["shot"] = None 
kwargs_dict["equil_time"] = 1900.0
kwargs_dict["vacuum_propagation_flag"] = False
kwargs_dict["Psi_BC_flag"] = False
kwargs_dict["poloidal_flux_enter"] = 1.0
kwargs_dict["poloidal_flux_zero_density"] = 1.0
kwargs_dict["poloidal_flux_zero_temperature"] = 1.0

kwargs_dict["auto_delta_sign"] = True
kwargs_dict["delta_X"] = 1e-3           # default is 1e-3
kwargs_dict["delta_Y"] = 1e-3           # default is 1e-3
kwargs_dict["delta_Z"] = 1e-3           # default is 1e-3
kwargs_dict["delta_K_X"] = 1e-2         # test is 1e-3
kwargs_dict["delta_K_Y"] = 1e-2         # test is 1e-3
kwargs_dict["delta_K_Z"] = 1e-2         # default is 1e-1
kwargs_dict["interp_order"] = 5
kwargs_dict["len_tau"] = 102
kwargs_dict["rtol"] = 1e-3              # test is 1e-5
kwargs_dict["atol"] = 1e-6              # test is 1e-10

kwargs_dict["ne_data_path"] = pathlib.Path(r"C:\Users\eduar\OneDrive\OneDrive - EWIKARTA001\OneDrive - Nanyang Technological University\University\2. To Be Uploaded\Year 3, PH4416 Professional Attachment\Scotty in 3d Cartesian")
kwargs_dict["magnetic_data_path"] = pathlib.Path(r"C:\Users\eduar\OneDrive\OneDrive - EWIKARTA001\OneDrive - Nanyang Technological University\University\2. To Be Uploaded\Year 3, PH4416 Professional Attachment\Scotty in 3d Cartesian")
kwargs_dict["Te_data_path"] = pathlib.Path(".")
kwargs_dict["output_path"] = pathlib.Path(r"C:\Users\eduar\OneDrive\OneDrive - EWIKARTA001\OneDrive - Nanyang Technological University\University\2. To Be Uploaded\Year 3, PH4416 Professional Attachment\Scotty in 3d Cartesian")
kwargs_dict["input_filename_suffix"] = "test"
kwargs_dict["output_filename_suffix"] = ""
kwargs_dict["figure_flag"] = True
kwargs_dict["detailed_analysis_flag"] = True

kwargs_dict["quick_run"] = False

# "plasmaLaunch_K_cartesian" is indexed as: K_X, K_Y, K_Z
kwargs_dict["plasmaLaunch_K_cartesian"] = np.array([freq_GHz_to_wavenumber(kwargs_dict["launch_freq_GHz"]) * np.cos(np.deg2rad(kwargs_dict["toroidal_launch_angle_Torbeam"])),
                                                    freq_GHz_to_wavenumber(kwargs_dict["launch_freq_GHz"]) * np.sin(np.deg2rad(kwargs_dict["toroidal_launch_angle_Torbeam"])),
                                                    0])
kwargs_dict["density_fit_method"] = "linear"
kwargs_dict["temperature_fit_method"] = None

kwargs_dict["B_T_axis"] = None
kwargs_dict["B_p_a"] = None
kwargs_dict["R_axis"] = None
kwargs_dict["minor_radius_a"] = None

# pre-calculations for plasmaLaunch_Psi_3D_lab_cartesian
stored = {"tor_angle": np.deg2rad(kwargs_dict["toroidal_launch_angle_Torbeam"]),
          "sin_tor_angle": np.sin(np.deg2rad(kwargs_dict["toroidal_launch_angle_Torbeam"])),
          "cos_tor_angle": np.cos(np.deg2rad(kwargs_dict["toroidal_launch_angle_Torbeam"])),
          "tan_tor_angle": np.tan(np.deg2rad(kwargs_dict["toroidal_launch_angle_Torbeam"])),
          "K_0": freq_GHz_to_wavenumber(kwargs_dict["launch_freq_GHz"]),
          "1/R_0": kwargs_dict["launch_beam_curvature"],
          "W_0": kwargs_dict["launch_beam_width"],
          "dne_dX": 1*10**19}

stored["L"] = ((constants.e**2) / ((2*constants.pi*kwargs_dict["launch_freq_GHz"]*(10**9))**2 * constants.epsilon_0 * constants.electron_mass) * stored["dne_dX"])**(-1)
stored["K_0/L"] = stored["K_0"] / stored["L"]

stored["Psi_YY0"] = 2j/stored["W_0"]**2
stored["Psi_Yg0"] = -(1/2)*stored["K_0/L"]*stored["sin_tor_angle"]
stored["Psi_YX0"] = 0
stored["Psi_gg0"] = -(1/2)*stored["K_0/L"]*stored["cos_tor_angle"]
stored["Psi_gX0"] = 0
stored["Psi_XX0"] = 2j/stored["W_0"]**2

stored["Psi_beamframe"] = np.array([
        [stored["Psi_YY0"], stored["Psi_Yg0"], stored["Psi_YX0"]],
        [stored["Psi_Yg0"], stored["Psi_gg0"], stored["Psi_gX0"]],
        [stored["Psi_YX0"], stored["Psi_gX0"], stored["Psi_XX0"]]])
stored["rot_matrix_lab2beam"] = np.array([
        [ stored["sin_tor_angle"],     stored["cos_tor_angle"],     0],
        [-stored["cos_tor_angle"],     stored["sin_tor_angle"],     0],
        [                       0,                           0,     1]])
stored["rot_matrix_beam2lab"] = stored["rot_matrix_lab2beam"].T
stored["Psi_labframe"] = np.matmul(stored["rot_matrix_beam2lab"].T, np.matmul(stored["Psi_beamframe"], stored["rot_matrix_beam2lab"]))

# Launching with Psi instead
kwargs_dict["plasmaLaunch_Psi_3D_lab_cartesian"] = stored["Psi_labframe"]

output = beam_me_up_3D(**kwargs_dict)

In [ ]:
# Plotting everything

import matplotlib.pyplot as plt
import numpy as np

# Extracting data from 3d Scotty
# data generated by 3d Scotty is returned point-wise
# we convert it to variable-wise (i.e. an array of X coords, K_x values, etc) for plotting
scotty_3d_data = {
    "X_coords": [],
    "Y_coords": [],
    "Z_coords": [],
    "K_X_values": [],
    "K_Y_values": [],
    "K_Z_values": [],
    "Psi_3D_cartesian": [],
    "tau_values": [],
    "dH_dX": [],
    "dH_dY": [],
    "dH_dZ": [],
    "dH_dKx": [],
    "dH_dKy": [],
    "dH_dKz": [],
    "d2H_dX2": [],
    "d2H_dY2": [],
    "d2H_dZ2": [],
    "d2H_dX_dY": [],
    "d2H_dX_dZ": [],
    "d2H_dY_dZ": [],
    "d2H_dKx2": [],
    "d2H_dKy2": [],
    "d2H_dKz2": [],
    "d2H_dKx_dKy": [],
    "d2H_dKx_dKz": [],
    "d2H_dKy_dKz": [],
    "d2H_dX_dKx": [],
    "d2H_dX_dKy": [],
    "d2H_dX_dKz": [],
    "d2H_dY_dKx": [],
    "d2H_dY_dKy": [],
    "d2H_dY_dKz": [],
    "d2H_dZ_dKx": [],
    "d2H_dZ_dKy": [],
    "d2H_dZ_dKz": [],
}
for point in range(len(output)):
    for i, key in enumerate(scotty_3d_data.keys()):
        scotty_3d_data[key].append(output[point][i])




# Extracting data from 2d Scotty
scotty_2d_data = {}
h5file_path = r"...\Scotty in 3d Cartesian\tokamak benchmark\scotty_output.h5"
import h5py
from scotty.fun_general import find_Psi_3D_lab_Cartesian
with h5py.File(h5file_path, "r") as h5file:
    print(h5file["solver_output"].keys())
    scotty_2d_data["tau_values"] = list(h5file["solver_output"]["tau"])

    _R    = np.array(list(h5file["solver_output"]["q_R"]))
    _zeta = np.array(list(h5file["solver_output"]["q_zeta"]))
    scotty_2d_data["X_coords"] = _R * np.cos(_zeta)
    scotty_2d_data["Y_coords"] = _R * np.sin(_zeta)
    scotty_2d_data["Z_coords"] = np.array(list(h5file["solver_output"]["q_Z"]))

    _K_R    = np.array(list(h5file["solver_output"]["K_R"]))
    _K_zeta = np.zeros_like(_K_R)
    _K_Z    = np.array(list(h5file["solver_output"]["K_Z"]))

    # scotty_2d_data["Psi_3D_cylindrical"] = list(h5file["solver_output"]["Psi_3D"][75:]) # REMEMBER to un-comment find_Psi_3D_lab_Cartesian below
    scotty_2d_data["Psi_3D_cartesian"] =  list(h5file["analysis"]["Psi_3D_Cartesian"])

    # Converting first derivatives from cylindrical to cartesian
    _dH_dR    = np.array((h5file["analysis"]["dH_dR"]))
    _dH_dzeta = np.zeros_like(_dH_dR)
    scotty_2d_data["dH_dX"] = _dH_dR*np.cos(_zeta) - 1/_R*np.sin(_zeta)*_dH_dzeta
    scotty_2d_data["dH_dY"] = _dH_dR*np.sin(_zeta) + 1/_R*np.cos(_zeta)
    scotty_2d_data["dH_dZ"] = np.array(h5file["analysis"]["dH_dZ"])
    
    _dH_dKR    = np.array((h5file["analysis"]["dH_dKR"]))
    _dH_dKzeta = np.array(_dH_dKR)
    scotty_2d_data["dH_dKx"] = _dH_dKR*np.cos(_zeta) - _dH_dKzeta*np.sin(_zeta)
    scotty_2d_data["dH_dKy"] = _dH_dKR*np.sin(_zeta) - _dH_dKzeta*np.cos(_zeta)
    scotty_2d_data["dH_dKz"] = np.array((h5file["analysis"]["dH_dKZ"]))

    # Getting second derivatives (cylindrical) from grad_grad_H, gradK_grad_H, gradK_gradK_H
    scotty_2d_data["grad_grad_H"] = np.array(h5file["analysis"]["grad_grad_H"])
    _d2H_dR2      = scotty_2d_data["grad_grad_H"][:,0,0] # top left each grad_grad_H
    _d2H_dR_dzeta = np.zeros_like(_d2H_dR2)
    _d2H_dR_dZ    = scotty_2d_data["grad_grad_H"][:,0,2] # top right of each grad_grad_H
    _d2H_dzeta2   = np.zeros_like(_d2H_dR2)
    _d2H_dzeta_dZ = np.zeros_like(_d2H_dR2)
    _d2H_dZ2      = scotty_2d_data["grad_grad_H"][:,2,2] # bottom right of each grad_grad_H

    scotty_2d_data["gradK_grad_H"] = np.array(h5file["analysis"]["gradK_grad_H"])
    _d2H_dKR_dR       = scotty_2d_data["gradK_grad_H"][:,0,0] # top left of each gradK_grad_H
    _d2H_dKR_dzeta    = np.zeros_like(_d2H_dKR_dR)
    _d2H_dKR_dZ       = scotty_2d_data["gradK_grad_H"][:,0,2] # top right of each gradK_grad_H
    _d2H_dKzeta_dR    = scotty_2d_data["gradK_grad_H"][:,1,0] # middle left of each gradK_grad_H
    _d2H_dKzeta_dzeta = np.zeros_like(_d2H_dKR_dR)
    _d2H_dKzeta_dZ    = scotty_2d_data["gradK_grad_H"][:,1,2] # middle right of each gradK_grad_H
    _d2H_dKZ_dR       = scotty_2d_data["gradK_grad_H"][:,2,0] # bottom left of each gradK_grad_H
    _d2H_dKZ_dzeta    = np.zeros_like(_d2H_dKR_dR)
    _d2H_dKZ_dZ       = scotty_2d_data["gradK_grad_H"][:,2,2] # bottom right of each gradK_grad_H

    scotty_2d_data["gradK_gradK_H"] = np.array(h5file["analysis"]["gradK_gradK_H"])
    _d2H_dKR2       = scotty_2d_data["gradK_gradK_H"][:,0,0] # top left of each gradK_gradK_H
    _d2H_dKR_dKzeta = scotty_2d_data["gradK_gradK_H"][:,0,1] # top middle of each gradK_gradK_H
    _d2H_dKR_dKZ    = scotty_2d_data["gradK_gradK_H"][:,0,2] # top right of each gradK_gradK_H
    _d2H_dKzeta2    = scotty_2d_data["gradK_gradK_H"][:,1,1] # middle middle of each gradK_gradK_H
    _d2H_dKzeta_dKZ = scotty_2d_data["gradK_gradK_H"][:,1,2] # middle right of each gradK_gradK_H
    _d2H_dKZ2       = scotty_2d_data["gradK_gradK_H"][:,2,2] # bottom right of each gradK_gradK_H

    # Converting second derivatives from cylindrical to cartesian
    _R2     = _R**2
    _sin    = np.sin(_zeta)
    _sin2   = np.sin(_zeta)**2
    _sincos = np.sin(_zeta)*np.cos(_zeta)
    _cos    = np.cos(_zeta)
    _cos2   = np.cos(_zeta)**2
    
    scotty_2d_data["d2H_dX2"]   = _cos2*_d2H_dR2 - 2*_sincos/_R*_d2H_dR_dzeta + _sin2/_R2*_d2H_dzeta2 + _sin2/R*_dH_dR + 2*_sincos/_R2*_dH_dzeta
    scotty_2d_data["d2H_dX_dY"] = _sincos*_d2H_dR2 + (_cos2-_sin2)/_R*_d2H_dR_dzeta - _sincos/_R2*_d2H_dzeta2 - _sincos/_R*_dH_dR + (_sin2-_cos2)/_R2*_dH_dzeta
    scotty_2d_data["d2H_dX_dZ"] = _cos*_d2H_dR_dZ - _sin/_R*_d2H_dzeta_dZ
    scotty_2d_data["d2H_dY2"]   = _sin2*_d2H_dR2 + 2*_sincos/_R*_d2H_dR_dzeta + _cos2/_R2*_d2H_dzeta2 + _cos2/_R*_dH_dR - 2*_sincos/_R2*_dH_dzeta
    scotty_2d_data["d2H_dY_dZ"] = _sin*_d2H_dR_dZ + _cos/_R*_d2H_dzeta_dZ
    scotty_2d_data["d2H_dZ2"]   = _d2H_dKZ2

    scotty_2d_data["d2H_dX_dKx"] = _sin2/_R2*_d2H_dzeta2 + _cos2*_d2H_dKR_dR - _sincos/_R*_d2H_dKR_dzeta - _sincos*_d2H_dKzeta_dR + _sin2/_R*_dH_dKR + _sincos/_R2*_dH_dzeta
    scotty_2d_data["d2H_dX_dKy"] = _sin2*_d2H_dKR_dR + _sincos/_R*_d2H_dKR_dzeta + _R*_sincos*_d2H_dKzeta_dR + _cos2*_d2H_dKzeta_dzeta + _cos2/_R*_dH_dKR
    scotty_2d_data["d2H_dX_dKz"] = _sin*_d2H_dR_dZ + _cos/_R*_d2H_dzeta_dZ
    scotty_2d_data["d2H_dY_dKx"] = _sincos*_d2H_dKR_dR + _cos2/_R*_d2H_dKR_dzeta - _R*_sin2*_d2H_dKzeta_dR - _sincos*_d2H_dKzeta_dzeta - _sincos/_R*_dH_dKR - _dH_dKzeta
    scotty_2d_data["d2H_dY_dKy"] = _sin2*_d2H_dKR_dR + _sincos/_R*_d2H_dKR_dzeta + _R*_sincos*_d2H_dKzeta_dR + _cos2*_d2H_dKzeta_dzeta + _cos2/_R*_dH_dKR
    scotty_2d_data["d2H_dY_dKz"] = _sin*_d2H_dR_dZ + _cos/_R*_d2H_dzeta_dZ
    scotty_2d_data["d2H_dZ_dKx"] = _cos*_d2H_dKR_dZ - _R*_sin*_d2H_dKzeta_dZ
    scotty_2d_data["d2H_dZ_dKy"] = _sin*_d2H_dKR_dZ + _R*_cos*_d2H_dKzeta_dZ
    scotty_2d_data["d2H_dZ_dKz"] = _d2H_dKZ_dZ

    scotty_2d_data["d2H_dKx2"]    = _cos2*_d2H_dKR2 - 2*_R*_sincos*_d2H_dKR_dKzeta + _R2*_sin*_d2H_dKzeta2 + _R*_sin2*_dH_dKR + 2*_R2*_sincos*_dH_dKzeta
    scotty_2d_data["d2H_dKx_dKy"] = _sincos*_d2H_dKR2 + _R*(_cos2-_sin2)*_d2H_dKR_dKzeta - _R2*_sincos*_d2H_dKzeta2 - _R*_sincos*_dH_dKR + _R2*(_sin2-_cos2)*_dH_dKzeta
    scotty_2d_data["d2H_dKx_dKz"] = _cos*_d2H_dKR_dKZ - _R*_sin*_d2H_dKzeta_dKZ
    scotty_2d_data["d2H_dKy2"]    = _sin2*_d2H_dKR2 + 2*_R*_sincos*_d2H_dKR_dKzeta + _R2*_cos2*_d2H_dKzeta2 + _R*_cos2*_dH_dKR - 2*_R2*_sincos*_dH_dKzeta
    scotty_2d_data["d2H_dKy_dKz"] = _sin*_d2H_dKR_dKZ + _R*_cos*_d2H_dKzeta_dKZ
    scotty_2d_data["d2H_dKz2"]    = _d2H_dKZ2





# Extracting data from analytical solution
sin_angle_temp = stored["sin_tor_angle"]
cos_angle_temp = stored["cos_tor_angle"]
tan_angle_temp = stored["tan_tor_angle"]
Y_array_temp = np.linspace(start=0, stop=1, num=100)

# Calculating the analytic solution (from Juan's paper)
def Psi_analytical_solution_doublecheck(K_x):
    tau_prime = stored["cos_tor_angle"] - K_x/stored["K_0"]
    tau = (tau_prime)*stored["K_0"]*stored["L"]
    Psi_0 = kwargs_dict["plasmaLaunch_Psi_3D_lab_cartesian"]
    Psi_0_inverse = np.linalg.inv(Psi_0)
    Psi_inverse = 2*(tau / (stored["K_0"])**2)*np.eye(3) + Psi_0_inverse
    Psi_lab_frame = np.linalg.inv(Psi_inverse)
    return Psi_lab_frame

analytic_soln = {}
analytic_soln["K_X_values"] = np.linspace(start=-kwargs_dict["plasmaLaunch_K_cartesian"][0], stop=kwargs_dict["plasmaLaunch_K_cartesian"][0], num=100)
analytic_soln["K_Y_values"] = np.ones_like(analytic_soln["K_X_values"]) * kwargs_dict["plasmaLaunch_K_cartesian"][1]
analytic_soln["K_Z_values"] = np.zeros_like(analytic_soln["K_X_values"])
analytic_soln["tau_values"] = stored["L"]*(stored["K_0"]*cos_angle_temp - analytic_soln["K_X_values"])
analytic_soln["X_coords"] = 2*cos_angle_temp/stored["K_0"]*analytic_soln["tau_values"] - analytic_soln["tau_values"]**2/(stored["L"]*stored["K_0"]**2)
analytic_soln["Y_coords"] = 2*sin_angle_temp/stored["K_0"]*analytic_soln["tau_values"]
analytic_soln["Z_coords"] = np.zeros_like(analytic_soln["X_coords"])

analytic_soln["dH_dX"] = np.ones_like(analytic_soln["K_X_values"]) * stored["L"]**(-1)
analytic_soln["dH_dY"] = np.zeros_like(analytic_soln["dH_dX"])
analytic_soln["dH_dZ"] = np.zeros_like(analytic_soln["dH_dX"])
analytic_soln["dH_dKx"] = 2*cos_angle_temp/stored["K_0"] - 2*analytic_soln["tau_values"] / (stored["L"]*stored["K_0"]**2)
analytic_soln["dH_dKy"] = np.ones_like(analytic_soln["dH_dKx"]) * 2 * sin_angle_temp / stored["K_0"]
analytic_soln["dH_dKz"] = np.zeros_like(analytic_soln["dH_dKx"])

analytic_soln_second_order_derivatives_dict = {}
analytic_soln_second_order_derivatives_dict["d2H_dX2"]     = np.zeros_like(scotty_3d_data["tau_values"])
analytic_soln_second_order_derivatives_dict["d2H_dY2"]     = np.zeros_like(analytic_soln_second_order_derivatives_dict["d2H_dX2"])
analytic_soln_second_order_derivatives_dict["d2H_dZ2"]     = np.zeros_like(analytic_soln_second_order_derivatives_dict["d2H_dX2"])
analytic_soln_second_order_derivatives_dict["d2H_dX_dY"]   = np.zeros_like(analytic_soln_second_order_derivatives_dict["d2H_dX2"])
analytic_soln_second_order_derivatives_dict["d2H_dX_dZ"]   = np.zeros_like(analytic_soln_second_order_derivatives_dict["d2H_dX2"])
analytic_soln_second_order_derivatives_dict["d2H_dY_dZ"]   = np.zeros_like(analytic_soln_second_order_derivatives_dict["d2H_dX2"])
analytic_soln_second_order_derivatives_dict["d2H_dKx2"]    = np.ones_like(analytic_soln_second_order_derivatives_dict["d2H_dX2"]) * 2 / stored["K_0"]**2
analytic_soln_second_order_derivatives_dict["d2H_dKy2"]    = np.ones_like(analytic_soln_second_order_derivatives_dict["d2H_dX2"]) * 2 / stored["K_0"]**2
analytic_soln_second_order_derivatives_dict["d2H_dKz2"]    = np.ones_like(analytic_soln_second_order_derivatives_dict["d2H_dX2"]) * 2 / stored["K_0"]**2
analytic_soln_second_order_derivatives_dict["d2H_dKx_dKy"] = np.zeros_like(analytic_soln_second_order_derivatives_dict["d2H_dX2"])
analytic_soln_second_order_derivatives_dict["d2H_dKx_dKz"] = np.zeros_like(analytic_soln_second_order_derivatives_dict["d2H_dX2"])
analytic_soln_second_order_derivatives_dict["d2H_dKy_dKz"] = np.zeros_like(analytic_soln_second_order_derivatives_dict["d2H_dX2"])
analytic_soln_second_order_derivatives_dict["d2H_dX_dKx"]  = np.zeros_like(analytic_soln_second_order_derivatives_dict["d2H_dX2"])
analytic_soln_second_order_derivatives_dict["d2H_dX_dKy"]  = np.zeros_like(analytic_soln_second_order_derivatives_dict["d2H_dX2"])
analytic_soln_second_order_derivatives_dict["d2H_dX_dKz"]  = np.zeros_like(analytic_soln_second_order_derivatives_dict["d2H_dX2"])
analytic_soln_second_order_derivatives_dict["d2H_dY_dKx"]  = np.zeros_like(analytic_soln_second_order_derivatives_dict["d2H_dX2"])
analytic_soln_second_order_derivatives_dict["d2H_dY_dKy"]  = np.zeros_like(analytic_soln_second_order_derivatives_dict["d2H_dX2"])
analytic_soln_second_order_derivatives_dict["d2H_dY_dKz"]  = np.zeros_like(analytic_soln_second_order_derivatives_dict["d2H_dX2"])
analytic_soln_second_order_derivatives_dict["d2H_dZ_dKx"]  = np.zeros_like(analytic_soln_second_order_derivatives_dict["d2H_dX2"])
analytic_soln_second_order_derivatives_dict["d2H_dZ_dKy"]  = np.zeros_like(analytic_soln_second_order_derivatives_dict["d2H_dX2"])
analytic_soln_second_order_derivatives_dict["d2H_dZ_dKz"]  = np.zeros_like(analytic_soln_second_order_derivatives_dict["d2H_dX2"])





# Plotting routine
""" RAY TRAJECTORY """
fig = plt.figure()
ax = plt.axes(projection = '3d')
ax.plot(scotty_3d_data["X_coords"], scotty_3d_data["Y_coords"], scotty_3d_data["Z_coords"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty")
ax.plot(scotty_2d_data["X_coords"], scotty_2d_data["Y_coords"], scotty_2d_data["Z_coords"], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty")
ax.plot(analytic_soln["X_coords"], analytic_soln["Y_coords"], analytic_soln["Z_coords"], color='green', linestyle='loosely dotted', linewidth=2, zorder=1, label="Analytic Soln")
ax.set_xlim(0,0.3)
ax.set_ylim(0,0.5)
ax.set_zlim(-0.01,0.01)
ax.set_xlabel("X coord")
ax.set_ylabel("Y coord")
ax.set_zlabel("Z coord")
ax.legend()
plt.show()



""" RAY TRAJECTORY, ONLY IN XY-PLANE """
plt.plot(scotty_3d_data["X_coords"], scotty_3d_data["Y_coords"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty")
plt.plot(scotty_2d_data["X_coords"], scotty_2d_data["Y_coords"], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty")
plt.plot(analytic_soln["X_coords"], analytic_soln["Y_coords"], color='green', linestyle='loosely dotted', linewidth=2, zorder=1, label="Analytic Soln")
plt.xlim(0,0.3)
plt.ylim(0,0.5)
plt.xlabel("X coord")
plt.ylabel("Y coord")
plt.legend()
plt.show()



""" POSITION """
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
axs = axs.flatten()
plt.tight_layout()
axs[0].plot(scotty_3d_data["tau_values"], scotty_3d_data["X_coords"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty")
axs[0].plot(scotty_2d_data["tau_values"], scotty_2d_data["X_coords"], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty")
axs[0].plot(analytic_soln["tau_values"], analytic_soln["X_coords"], color='green', linestyle='loosely dotted', linewidth=2, zorder=1, label="Analytic Soln")
axs[0].set_title("X coord vs tau")
axs[0].legend(loc="best")
axs[0].grid(True, which="both")
axs[1].plot(scotty_3d_data["tau_values"], scotty_3d_data["Y_coords"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty")
axs[1].plot(scotty_2d_data["tau_values"], scotty_2d_data["Y_coords"], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty")
axs[1].plot(analytic_soln["tau_values"], analytic_soln["Y_coords"], color='green', linestyle='loosely dotted', linewidth=2, zorder=1, label="Analytic Soln")
axs[1].set_title("Y coord vs tau")
axs[1].legend(loc="best")
axs[1].grid(True, which="both")
axs[2].plot(scotty_3d_data["tau_values"], scotty_3d_data["Z_coords"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty")
axs[2].plot(scotty_2d_data["tau_values"], scotty_2d_data["Z_coords"], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty")
axs[2].plot(analytic_soln["tau_values"], analytic_soln["Z_coords"], color='green', linestyle='loosely dotted', linewidth=2, zorder=1, label="Analytic Soln")
axs[2].set_title("Z coord vs tau")
axs[2].legend(loc="best")
axs[2].grid(True, which="both")
plt.show()



""" WAVEVECTORS """
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
axs = axs.flatten()
plt.tight_layout()
axs[0].plot(scotty_3d_data["tau_values"], scotty_3d_data["K_X_values"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty")
axs[0].plot(scotty_2d_data["tau_values"], scotty_2d_data["K_X_values"], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty")
axs[0].plot(analytic_soln["tau_values"], analytic_soln["K_X_values"], color='green', linestyle='loosely dotted', linewidth=2, zorder=1, label="Analytic Soln")
axs[0].set_title("K_X vs tau")
axs[0].legend(loc="best")
axs[0].grid(True, which="both")
axs[1].plot(scotty_3d_data["tau_values"], scotty_3d_data["K_Y_values"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty")
axs[1].plot(scotty_2d_data["tau_values"], scotty_2d_data["K_Y_values"], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty")
axs[1].plot(analytic_soln["tau_values"], analytic_soln["K_Y_values"], color='green', linestyle='loosely dotted', linewidth=2, zorder=1, label="Analytic Soln")
axs[1].set_title("K_Y vs tau")
axs[1].legend(loc="best")
axs[1].grid(True, which="both")
axs[2].plot(scotty_3d_data["tau_values"], scotty_3d_data["K_Z_values"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty")
axs[2].plot(scotty_2d_data["tau_values"], scotty_2d_data["K_Z_values"], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty")
axs[2].plot(analytic_soln["tau_values"], analytic_soln["K_Z_values"], color='green', linestyle='loosely dotted', linewidth=2, zorder=1, label="Analytic Soln")
axs[2].set_title("K_Z vs tau")
axs[2].legend(loc="best")
axs[2].grid(True, which="both")
plt.show()



""" FIRST DERIVATIVES """
fig, axs = plt.subplots(2, 3, figsize=(20, 10))
axs = axs.flatten()
plt.tight_layout()
counter = 0
for element in ["dH_dX", "dH_dY", "dH_dZ", "dH_dKx", "dH_dKy", "dH_dKz"]:
    axs[counter].plot(scotty_3d_data["tau_values"], scotty_3d_data[f"{element}"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty")
    axs[counter].plot(scotty_2d_data["tau_values"], scotty_2d_data[f"{element}"], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty")
    axs[counter].plot(analytic_soln["tau_values"], analytic_soln[f"{element}"], color='green', linestyle='loosely dotted', linewidth=2, zorder=1, label="Analytic Soln")
    axs[counter].set_title(f"{element} vs tau")
    axs[counter].grid(True, which="both")
    axs[counter].legend(loc="best")
    counter += 1
plt.show()



""" SECOND DERIVATIVES """
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
def plot_how_second_derivatives_evolve(dictionary):
    counter = 0
    for key, value in dictionary.items():
        # xmin, xmax = 0, 1002 # 0, 900
        # ymin, ymax = min(value[xmin:xmax+1]), max(value[xmin:xmax+1])
        # axs[counter].set_xlim(xmin, xmax)
        # axs[counter].set_ylim(ymin, ymax)
        axs[counter].plot(scotty_3d_data["tau_values"], scotty_3d_data[key], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty")
        axs[counter].plot(scotty_2d_data["tau_values"], scotty_2d_data[key], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty")
        axs[counter].plot(scotty_3d_data["tau_values"], analytic_soln_second_order_derivatives_dict[key], color='green', linestyle='loosely dotted', linewidth=2, zorder=1, label="Analytic Soln")
        axs[counter].set_xlabel("tau")
        axs[counter].set_ylabel(key)
        axs[counter].set_title(f"{key} changing along tau")
        axs[counter].grid(True, which="both", axis="x")
        axs[counter].legend(loc="best")
        counter += 1

fig, axs = plt.subplots(7, 3, figsize=(15, 15))
axs = axs.flatten()
plot_how_second_derivatives_evolve(analytic_soln_second_order_derivatives_dict)
plt.tight_layout()
plt.show()

#  Tokamak Benchmark

In [ ]:
# constructing the data needed -- magnetic field data, ne data, polflux data

import json
import numpy as np
import scipy.constants as const
from scotty.fun_general import find_q_lab
from scipy.interpolate import RectBivariateSpline

# Data location for B field and ne/polflux
topfile_path = r"C:\Users\eduar\OneDrive\OneDrive - EWIKARTA001\OneDrive - Nanyang Technological University\University\2. To Be Uploaded\Year 3, PH4416 Professional Attachment\Scotty in 3d Cartesian\tokamak benchmark\topfile_189998_3000ms_quinn.json"
nefile_path  = r"C:\Users\eduar\OneDrive\OneDrive - EWIKARTA001\OneDrive - Nanyang Technological University\University\2. To Be Uploaded\Year 3, PH4416 Professional Attachment\Scotty in 3d Cartesian\tokamak benchmark\ne_189998_3000ms_quinn.dat"

with open(topfile_path, "r") as topfile:
    topfile_data = json.load(topfile)

R_coordinates  = topfile_data.get("R", [])
Z_coordinates  = topfile_data.get("Z", [])
Br_field = topfile_data.get("Br", [])
Bt_field = topfile_data.get("Bt", [])
Bz_field = topfile_data.get("Bz", [])
polflux_field = topfile_data.get("pol_flux", [])

num_R_points = len(R_coordinates)  # R has 129 data points in total
min_R = min(R_coordinates) # 0.84
max_R = max(R_coordinates) # 2.54

num_Z_points = len(Z_coordinates)  # Z has 129 data points in total also
min_Z = min(Z_coordinates) # -1.6
max_Z = max(Z_coordinates) # 1.6

# This means X and Z should now have 129 data points.
# The Y (toroidal direction) should have just enough points for interpolation.
# So maybe 20 points or so?
X = np.linspace(start=min_R, stop=max_R, num=num_R_points+1)
Y = np.linspace(start=-0.1,  stop=0.1,   num=21)
Z = np.linspace(start=min_Z, stop=max_Z, num=num_Z_points+1)

# Creating the interpolators
Br_field_spline_class = RectBivariateSpline(R_coordinates, Z_coordinates, Br_field, kx=5, ky=5)
Bt_field_spline_class = RectBivariateSpline(R_coordinates, Z_coordinates, Bt_field, kx=5, ky=5)
Bz_field_spline_class = RectBivariateSpline(R_coordinates, Z_coordinates, Bz_field, kx=5, ky=5)
polflux_field_spline_class = RectBivariateSpline(R_coordinates, Z_coordinates, polflux_field, kx=5, ky=5)

# Calculate the value of Bx, By, Bz, polflux at a particular coordinate
def calculate_values(X_coord, Y_coord, Z_coord):
    R_coord = np.sqrt(X_coord**2 + Y_coord**2)
    zeta_coord = np.arctan(Y_coord / X_coord)

    Br_value = Br_field_spline_class(R_coord, Z_coord)
    Bt_value = Bt_field_spline_class(R_coord, Z_coord)
    Bz_value = Bz_field_spline_class(R_coord, Z_coord)
    polflux_value = polflux_field_spline_class(R_coord, Z_coord)

    Bx_value = (Br_value*X_coord - Bt_value*Y_coord) / R_coord
    By_value = (Br_value*Y_coord + Bt_value*X_coord) / R_coord

    return Bx_value, By_value, Bz_value, polflux_value

# Initialise empty arrays to put data into
zero_array = np.zeros((len(Z), len(Y), len(X)))
Bx_array = np.copy(zero_array)
By_array = np.copy(zero_array)
Bz_array = np.copy(zero_array)
polflux_array = np.copy(zero_array)

for i in range(len(X)):
    for j in range(len(Y)):
        for k in range(len(Z)):
            X_coord = X[i]
            Y_coord = Y[j]
            Z_coord = Z[k]
            Bx_value, By_value, Bz_value, polflux_value = calculate_values(X_coord, Y_coord, Z_coord)

            Bx_array[k][j][i] = float(Bx_value)
            By_array[k][j][i] = float(By_value)
            Bz_array[k][j][i] = float(Bz_value)
            polflux_array[k][j][i] = float(polflux_value)





# Writing the data into files that 3D Scotty can read
topfile_data = {
    "X": X.tolist(),
    "Y": Y.tolist(),
    "Z": Z.tolist(),
    "Bx": Bx_array.tolist(),
    "By": By_array.tolist(),
    "Bz": Bz_array.tolist(),
    "pol_flux": polflux_array.tolist()
}

with open("topfile_for3dscotty.json", "w") as topfile:
    json.dump(topfile_data, topfile, indent=4)

In [ ]:
# Checking if the data that was just generated is correct and that it is read correctly
# Create another spline to read the data that was just written

import numpy as np
import pathlib
import scipy.constants as const

from scotty.beam_me_up_3D_temp import create_magnetic_geometry_3D
scotty_3d_topfile_path_without_filename = r"C:\Users\eduar\OneDrive\OneDrive - EWIKARTA001\OneDrive - Nanyang Technological University\University\2. To Be Uploaded\Year 3, PH4416 Professional Attachment\Scotty in 3d Cartesian\tokamak benchmark\test files"
field_3d_scotty = create_magnetic_geometry_3D(
    find_B_method = "omfit_3D",
    magnetic_data_path = pathlib.Path(scotty_3d_topfile_path_without_filename),
    input_filename_suffix = "",
    interp_order = 5
)

from scotty.beam_me_up import create_magnetic_geometry
scotty_2d_topfile_path = r"C:\Users\eduar\OneDrive\OneDrive - EWIKARTA001\OneDrive - Nanyang Technological University\University\2. To Be Uploaded\Year 3, PH4416 Professional Attachment\Scotty in 3d Cartesian\tokamak benchmark"
field_2d_scotty = create_magnetic_geometry(
    find_B_method = "omfit",
    magnetic_data_path = pathlib.Path(scotty_2d_topfile_path),
    input_filename_suffix = "_189998_3000ms_quinn",
    interp_order = 5
)

rand_R = np.random.uniform(0.84, 2.56)
rand_zeta = np.random.uniform(0, 2*const.pi)

rand_X = rand_R*np.cos(rand_zeta)
rand_Y = rand_R*np.sin(rand_zeta)
rand_Z = np.random.uniform(-1.6, 1.6)

print(field_3d_scotty.B_X(rand_X, rand_Y, rand_Z))
print(field_2d_scotty.B_R(rand_R, rand_Z))
print()
print(field_3d_scotty.B_Y(rand_X, rand_Y, rand_Z))
print(field_2d_scotty.B_T(rand_R, rand_Z))
print()
print(field_3d_scotty.B_Z(rand_X, rand_Y, rand_Z))
print(field_2d_scotty.B_Z(rand_R, rand_Z))
print()
print(field_3d_scotty.polflux(rand_X, rand_Y, rand_Z))
print(field_2d_scotty.poloidal_flux(rand_R, rand_Z))

In [ ]:
# the actual beam tracing code

import numpy as np
import os
import pathlib
from scotty.beam_me_up_3D_temp import beam_me_up_3D

kwargs_dict = {
    'poloidal_launch_angle_Torbeam': 0.0,
    'toroidal_launch_angle_Torbeam': 0.0,
    'launch_freq_GHz': 72.5,
    'mode_flag': -1,
    'launch_beam_width': 0.1265,
    'launch_beam_curvature': -0.95,
    'find_B_method': "omfit_3D",
    'Psi_BC_flag': 'discontinuous', # NOT USED FOR 3D, BUT PUT HERE FOR REFERENCE
    'poloidal_flux_enter': 0.95,
    'poloidal_flux_zero_density': 1.0,
    'figure_flag': True, # NOT USED FOR 3D, BUT PUT HERE FOR REFERENCE
    'vacuum_propagation_flag': True, # NOT USED FOR 3D, BUT PUT HERE FOR REFERENCE
    'vacuumLaunch_flag': False,       # actual value is True, but False used for my 3D Scotty
    'ne_data_path': pathlib.Path(r"C:\Users\eduar\OneDrive\OneDrive - EWIKARTA001\OneDrive - Nanyang Technological University\University\2. To Be Uploaded\Year 3, PH4416 Professional Attachment\Scotty in 3d Cartesian\tokamak benchmark\test files"),
    'magnetic_data_path': pathlib.Path(r"C:\Users\eduar\OneDrive\OneDrive - EWIKARTA001\OneDrive - Nanyang Technological University\University\2. To Be Uploaded\Year 3, PH4416 Professional Attachment\Scotty in 3d Cartesian\tokamak benchmark\test files"),
    'input_filename_suffix': "",
    'output_path': pathlib.Path(r"C:\Users\eduar\OneDrive\OneDrive - EWIKARTA001\OneDrive - Nanyang Technological University\University\2. To Be Uploaded\Year 3, PH4416 Professional Attachment\Scotty in 3d Cartesian\tokamak benchmark\test files"),
}

kwargs_dict["delta_X"] = -0.00001
kwargs_dict["delta_Y"] = -0.00001
kwargs_dict["delta_Z"] = -0.00001
kwargs_dict["delta_K_X"] = 0.1
kwargs_dict["delta_K_Y"] = 0.1
kwargs_dict["delta_K_Z"] = 0.1
kwargs_dict["interp_smoothing"] = 0
kwargs_dict["interp_order"] = 5
kwargs_dict["len_tau"] = 1002
kwargs_dict["rtol"] = 1e-4
kwargs_dict["atol"] = 1e-7





# CHECKED AND CONFIRMED CORRECT
if kwargs_dict['poloidal_launch_angle_Torbeam'] == 0 and kwargs_dict['toroidal_launch_angle_Torbeam'] == 0 and kwargs_dict['poloidal_flux_enter'] == 0.95:
    tau_adjustment = 41.15778660622825
    kwargs_dict["launch_position_cartesian"] = np.array([2.2621936543069086, -0.00040362814170096634, -0.09163904444906242])
    kwargs_dict["plasmaLaunch_K_cartesian"] = np.array([-900.6297175486496, 0.16069335998831755, -50.543485174031446])
    kwargs_dict["plasmaLaunch_Psi_3D_lab_cartesian"] = np.array(
[[-3.46342996e+03   +5.53335966j,     -2.28386497e+00  -19.98312296j,      4.33645205e+02  -88.91438058j],
 [-2.28386497e+00  -19.98312296j,     -4.92365259e+03+1851.21452832j,      1.40497528e+03 -146.34738761j],
 [ 4.33645205e+02  -88.91438058j,      1.40497528e+03 -146.34738761j,     -3.88631484e+03+1551.13527391j]]
)

# CHECKED AND CONFIRMED CORRECT
elif kwargs_dict['poloidal_launch_angle_Torbeam'] == -7.0 and kwargs_dict['toroidal_launch_angle_Torbeam'] == 0 and kwargs_dict['poloidal_flux_enter'] == 0.95:
    tau_adjustment = 40.031518240219356
    kwargs_dict["launch_position_cartesian"] = np.array([2.26436572068288, 0.0009066122128788577, 0.005151249838071285])
    kwargs_dict["plasmaLaunch_K_cartesian"] = np.array([-879.5700312013402, -0.3521643721619699, 228.99532936732646])
    kwargs_dict["plasmaLaunch_Psi_3D_lab_cartesian"] = np.array(
[[-3.85542237e+03 +115.51896532j,     -2.98291792e+00  +53.18414012j,     -1.25590394e+03 +425.70495113j],
 [-2.98291792e+00  +53.18414012j,     -4.93287638e+03+1881.21400333j,      8.01285958e+02  -65.63402721j],
 [-1.25590394e+03 +425.70495113j,      8.01285958e+02  -65.63402721j,     -3.95898499e+03+1605.52821159j]]
)

# CHECKED AND CONFIRMED CORRECT
elif kwargs_dict['poloidal_launch_angle_Torbeam'] == -10.0 and kwargs_dict['toroidal_launch_angle_Torbeam'] == 0 and kwargs_dict['poloidal_flux_enter'] == 0.95:
    tau_adjustment = 37.11475407679377
    kwargs_dict["launch_position_cartesian"] = np.array([2.263427097581486, 0.0013114878516376572, 0.04728357416749444])
    kwargs_dict["plasmaLaunch_K_cartesian"] = np.array([-846.3452459490945, -0.49039419451130006, 355.618568463616])
    kwargs_dict["plasmaLaunch_Psi_3D_lab_cartesian"] = np.array(
[[-4611.46076077 +329.02434708j,      716.67508628    +3.04269807j,     -2247.71067532 +763.45958508j],
 [  716.67508628   +3.04269807j,     -5125.6015488  +1941.2629018j,      2510.80973791 -271.37972463j],
 [-2247.71067532 +763.45958508j,      2510.80973791 -271.37972463j,     -4529.76072795+1811.21688397j]]
)

# CHECKED AND CONFIRMED CORRECT
elif kwargs_dict['poloidal_launch_angle_Torbeam'] == -14.0 and kwargs_dict['toroidal_launch_angle_Torbeam'] == 0 and kwargs_dict['poloidal_flux_enter'] == 0.95:
    tau_adjustment = 30.613455225966494
    kwargs_dict["launch_position_cartesian"] = np.array([2.2598765012509063, 0.0014453779662245668, 0.10396136575632378])
    kwargs_dict["plasmaLaunch_K_cartesian"] = np.array([-780.9060774194234, -0.49945403537234, 526.2866721978481])
    kwargs_dict["plasmaLaunch_Psi_3D_lab_cartesian"] = np.array(
[[-12826.48959136+1032.60551938j,      2735.24430304 -161.67060233j,     -5353.52552418+1524.56699891j],
 [  2735.24430304 -161.67060233j,     -5663.29793949+2100.58972225j,      4867.72711371 -523.73792424j],
 [ -5353.52552418+1524.56699891j,      4867.72711371 -523.73792424j,     -5448.62479053+2289.76387434j]]
)

# CHECKED AND CONFIRMED CORRECT
elif kwargs_dict['poloidal_launch_angle_Torbeam'] == -30.0 and kwargs_dict['toroidal_launch_angle_Torbeam'] == 0 and kwargs_dict['poloidal_flux_enter'] == 0.95:
    tau_adjustment = 2.749489158455453
    kwargs_dict["launch_position_cartesian"] = np.array([2.1582516073680833, 7.719065632782897e-05, 0.40429184157427406])
    kwargs_dict["plasmaLaunch_K_cartesian"] = np.array([-1028.7420882715885, -0.03679333619613622, 925.4406379415204])
    kwargs_dict["plasmaLaunch_Psi_3D_lab_cartesian"] = np.array(
[[-61326.15525778+14822.71135012j,      40975.47999299 -3143.52639597j,     -40581.38383975+16547.45263864j],
 [ 40975.47999299 -3143.52639597j,     -14961.9685102 +12850.43919523j,      46188.12058581 -3974.36012186j],
 [-40581.38383975+16547.45263864j,      46188.12058581 -3974.36012186j,     -30082.00159944+18490.87760416j]]
)

# CHECKED AND CONFIRMED CORRECT
elif kwargs_dict['poloidal_launch_angle_Torbeam'] == 0.0 and kwargs_dict['toroidal_launch_angle_Torbeam'] == -5.0 and kwargs_dict['poloidal_flux_enter'] == 0.95:
    tau_adjustment = 45.6074282427938
    kwargs_dict["launch_position_cartesian"] = np.array([2.2588437570598807, 0.07118167483595075, -0.09124676478754781])
    kwargs_dict["plasmaLaunch_K_cartesian"] = np.array([-866.0699888916636, -27.29197720815518, -53.246064794769715])
    kwargs_dict["plasmaLaunch_Psi_3D_lab_cartesian"] = np.array(
[[-4126.84890536 +167.22088725j,     -1487.56726832 +577.62485042j,      888.06005508  -133.39949474j],
 [-1487.56726832 +577.62485042j,     -5125.31241745+2028.42094287j,      1610.41748905 -234.33463987j],
 [  888.06005508 -133.39949474j,      1610.41748905 -234.33463987j,     -4037.42236745+1641.40438372j]]
)

# CHECKED AND CONFIRMED CORRECT
elif kwargs_dict['poloidal_launch_angle_Torbeam'] == 0.0 and kwargs_dict['toroidal_launch_angle_Torbeam'] == -2.0 and kwargs_dict['poloidal_flux_enter'] == 0.95:
    tau_adjustment = 41.89277831967363
    kwargs_dict["launch_position_cartesian"] = np.array([2.2615909877048415, 0.02790475466858362, -0.09144859663924958])
    kwargs_dict["plasmaLaunch_K_cartesian"] = np.array([-895.5320907436455, -11.049567948360856, -51.14373952138453])
    kwargs_dict["plasmaLaunch_Psi_3D_lab_cartesian"] = np.array(
[[-3578.93064121  +24.65627082j,     -559.484711    +197.00042889j,      589.03899313   -96.87791499j],
 [ -559.484711   +197.00042889j,     -4959.1988885 +1881.08427337j,      1460.06895957 -170.16575875j],
 [  589.03899313  -96.87791499j,      1460.06895957 -170.16575875j,     -3927.92811746+1566.83089569j]]
)
    
# CHECKED AND CONFIRMED CORRECT
elif kwargs_dict['poloidal_launch_angle_Torbeam'] == 0.0 and kwargs_dict['toroidal_launch_angle_Torbeam'] == 2.0 and kwargs_dict['poloidal_flux_enter'] == 0.95:
    tau_adjustment = 43.7355107876609
    kwargs_dict["launch_position_cartesian"] = np.array([2.2602834349269463, -0.028885589285448147, -0.09199091518721854])
    kwargs_dict["plasmaLaunch_K_cartesian"] = np.array([-888.9423280386549, 11.360355338358094, -50.8966451703207])
    kwargs_dict["plasmaLaunch_Psi_3D_lab_cartesian"] = np.array(
[[-3631.3874589   +38.22126228j,      560.13220625  -242.92127249j,      303.22662391   -89.87771906j],
 [  560.13220625 -242.92127249j,     -4992.62393836+1916.82648729j,      1402.95128259 -143.57799329j],
 [  303.22662391  -89.87771906j,      1402.95128259 -143.57799329j,     -3883.83331825+1580.74075838j]]
)

# CHECKED AND CONFIRMED CORRECT
elif kwargs_dict['poloidal_launch_angle_Torbeam'] == 0 and kwargs_dict['toroidal_launch_angle_Torbeam'] == 5.0 and kwargs_dict['poloidal_flux_enter'] == 0.95:
    tau_adjustment = 48.00885946028994
    kwargs_dict["launch_position_cartesian"] = np.array([2.2573739496260554, -0.07244943850334908, -0.09257268650012045])
    kwargs_dict["plasmaLaunch_K_cartesian"] = np.array([-855.606915560603, 27.460333110611444, -52.17826188610897])
    kwargs_dict["plasmaLaunch_Psi_3D_lab_cartesian"] = np.array(
[[-4240.76190939 +206.20818307j,      1487.95189791  -633.3132573j,      106.2658659    -96.50255403j],
 [ 1487.95189791  -633.3132573j,     -5148.30666812+2064.93410939j,      1422.98399696 -142.32152851j],
 [  106.2658659   -96.50255403j,      1422.98399696 -142.32152851j,     -3892.78016806+1647.79533968j]]
)

# CHECKED AND CONFIRMED CORRECT
elif kwargs_dict['poloidal_launch_angle_Torbeam'] == -7.0 and kwargs_dict['toroidal_launch_angle_Torbeam'] == -5.0 and kwargs_dict['poloidal_flux_enter'] == 0.95:
    tau_adjustment = 43.93833026805542
    kwargs_dict["launch_position_cartesian"] = np.array([2.2619225021163003, 0.07184286103791511, 0.00675604439611002])
    kwargs_dict["plasmaLaunch_K_cartesian"] = np.array([-839.0368333797272, -26.649368654228436, 230.7971373803134])
    kwargs_dict["plasmaLaunch_Psi_3D_lab_cartesian"] = np.array(
[[-4547.48397672 +366.64578149j,     -1473.43107381 +659.28972869j,     -1142.01707418 +492.11619465j],
 [-1473.43107381 +659.28972869j,     -5059.52348833+2009.18000846j,      796.75628559   -32.12703643j],
 [-1142.01707418 +492.11619465j,      796.75628559   -32.12703643j,     -3966.81415554+1681.04432977j]]
)

# CHECKED AND CONFIRMED CORRECT
elif kwargs_dict['poloidal_launch_angle_Torbeam'] == -7.0 and kwargs_dict['toroidal_launch_angle_Torbeam'] == 5.0 and kwargs_dict['poloidal_flux_enter'] == 0.95:
    tau_adjustment = 40.89177224672123
    kwargs_dict["launch_position_cartesian"] = np.array([2.263408485469987, -0.06962509886786736, 0.005224487328000113])
    kwargs_dict["plasmaLaunch_K_cartesian"] = np.array([-857.1918340615352, 26.368225876324846, 231.1055014836462])
    kwargs_dict["plasmaLaunch_Psi_3D_lab_cartesian"] = np.array(
[[-4354.17832504 +250.19205958j,      1479.92798758 -536.77396034j,     -1513.29526526 +447.09058671j],
 [ 1479.92798758 -536.77396034j,     -5049.85407458+1975.36810526j,      903.2253801   -119.01688604j],
 [-1513.29526526 +447.09058671j,      903.2253801   -119.01688604j,     -4040.12962118+1655.64858978j]]
)

else:
    print("Need to find initial conditions from 2D Scotty first!")
    exit()

output = beam_me_up_3D(**kwargs_dict)



# Just a template for me to use
# elif kwargs_dict['poloidal_launch_angle_Torbeam'] == _ and kwargs_dict['toroidal_launch_angle_Torbeam'] == _ and kwargs_dict['poloidal_flux_enter'] == 0.95:
#     tau_adjustment = 40 # not exact
#     kwargs_dict["launch_position_cartesian"] = np.array()
#     kwargs_dict["plasmaLaunch_K_cartesian"] = np.array()
#     kwargs_dict["plasmaLaunch_Psi_3D_lab_cartesian"] = np.array(

# )

In [ ]:
# data generated by 3d Scotty is returned point-wise
# we convert it to variable-wise (i.e. an array of X coords, K_x values, etc) for plotting
import matplotlib.pyplot as plt
scotty_3d_data = {
    "X_coords": [],
    "Y_coords": [],
    "Z_coords": [],
    "K_X_values": [],
    "K_Y_values": [],
    "K_Z_values": [],
    "Psi_3D_cartesian": [],
    "tau_values_unadjusted": [],
    "dH_dX": [],
    "dH_dY": [],
    "dH_dZ": [],
    "dH_dKx": [],
    "dH_dKy": [],
    "dH_dKz": [],
    "d2H_dX2": [],
    "d2H_dY2": [],
    "d2H_dZ2": [],
    "d2H_dX_dY": [],
    "d2H_dX_dZ": [],
    "d2H_dY_dZ": [],
    "d2H_dKx2": [],
    "d2H_dKy2": [],
    "d2H_dKz2": [],
    "d2H_dKx_dKy": [],
    "d2H_dKx_dKz": [],
    "d2H_dKy_dKz": [],
    "d2H_dX_dKx": [],
    "d2H_dX_dKy": [],
    "d2H_dX_dKz": [],
    "d2H_dY_dKx": [],
    "d2H_dY_dKy": [],
    "d2H_dY_dKz": [],
    "d2H_dZ_dKx": [],
    "d2H_dZ_dKy": [],
    "d2H_dZ_dKz": [],
}
for point in range(len(output)):
    for i, key in enumerate(scotty_3d_data.keys()):
        scotty_3d_data[key].append(output[point][i])

# we need to adjust the tau values for the 3d output (it starts at '0')
# the tau values for the 2d output start at 40 something because we launch the ray from slightly deeper in the plasma due to poor derivative calculations on the outside (which correspond to small and large tau values)
scotty_3d_data["tau_values"] = np.array(scotty_3d_data["tau_values_unadjusted"]) + tau_adjustment # this value corresponds to the "first" corresponding tau for 2d Scotty

# now we find the elements of Psi individually
scotty_3d_data["Psi_3D_cartesian_XX_Re"] = [np.real(scotty_3d_data["Psi_3D_cartesian"][i][0][0]) for i in range(len(scotty_3d_data["tau_values"]))]
scotty_3d_data["Psi_3D_cartesian_XY_Re"] = [np.real(scotty_3d_data["Psi_3D_cartesian"][i][0][1]) for i in range(len(scotty_3d_data["tau_values"]))]
scotty_3d_data["Psi_3D_cartesian_XZ_Re"] = [np.real(scotty_3d_data["Psi_3D_cartesian"][i][0][2]) for i in range(len(scotty_3d_data["tau_values"]))]
scotty_3d_data["Psi_3D_cartesian_YY_Re"] = [np.real(scotty_3d_data["Psi_3D_cartesian"][i][1][1]) for i in range(len(scotty_3d_data["tau_values"]))]
scotty_3d_data["Psi_3D_cartesian_YZ_Re"] = [np.real(scotty_3d_data["Psi_3D_cartesian"][i][1][2]) for i in range(len(scotty_3d_data["tau_values"]))]
scotty_3d_data["Psi_3D_cartesian_ZZ_Re"] = [np.real(scotty_3d_data["Psi_3D_cartesian"][i][2][2]) for i in range(len(scotty_3d_data["tau_values"]))]
scotty_3d_data["Psi_3D_cartesian_XX_Im"] = [np.imag(scotty_3d_data["Psi_3D_cartesian"][i][0][0]) for i in range(len(scotty_3d_data["tau_values"]))]
scotty_3d_data["Psi_3D_cartesian_XY_Im"] = [np.imag(scotty_3d_data["Psi_3D_cartesian"][i][0][1]) for i in range(len(scotty_3d_data["tau_values"]))]
scotty_3d_data["Psi_3D_cartesian_XZ_Im"] = [np.imag(scotty_3d_data["Psi_3D_cartesian"][i][0][2]) for i in range(len(scotty_3d_data["tau_values"]))]
scotty_3d_data["Psi_3D_cartesian_YY_Im"] = [np.imag(scotty_3d_data["Psi_3D_cartesian"][i][1][1]) for i in range(len(scotty_3d_data["tau_values"]))]
scotty_3d_data["Psi_3D_cartesian_YZ_Im"] = [np.imag(scotty_3d_data["Psi_3D_cartesian"][i][1][2]) for i in range(len(scotty_3d_data["tau_values"]))]
scotty_3d_data["Psi_3D_cartesian_ZZ_Im"] = [np.imag(scotty_3d_data["Psi_3D_cartesian"][i][2][2]) for i in range(len(scotty_3d_data["tau_values"]))]










# data generated (B field, ne, polflux) by 2d Scotty is in the h5 file, and in cylindrical
# the lists below are sliced starting from index 75 because the derivatives at the start are quite ugly
# so, in 3d Scotty, we elect to launch the wave from a start point which corresponds to the 75th tau (arbitrary)
# that is produced by 2d Scotty
scotty_2d_data = {}
h5file_path = r"C:\Users\eduar\OneDrive\OneDrive - EWIKARTA001\OneDrive - Nanyang Technological University\University\2. To Be Uploaded\Year 3, PH4416 Professional Attachment\Scotty in 3d Cartesian\tokamak benchmark\scotty_output.h5"
import h5py
from scotty.fun_general import find_Psi_3D_lab_Cartesian
with h5py.File(h5file_path, "r") as h5file:
    print(h5file["solver_output"].keys())
    scotty_2d_data["tau_values"] = list(h5file["solver_output"]["tau"])

    _R    = np.array(list(h5file["solver_output"]["q_R"]))
    _zeta = np.array(list(h5file["solver_output"]["q_zeta"]))
    scotty_2d_data["X_coords"] = _R * np.cos(_zeta)
    scotty_2d_data["Y_coords"] = _R * np.sin(_zeta)
    scotty_2d_data["Z_coords"] = np.array(list(h5file["solver_output"]["q_Z"]))

    _R2     = _R**2
    _sin    = np.sin(_zeta)
    _sin2   = np.sin(_zeta)**2
    _sincos = np.sin(_zeta)*np.cos(_zeta)
    _cos    = np.cos(_zeta)
    _cos2   = np.cos(_zeta)**2

    _K_R    = np.array(list(h5file["solver_output"]["K_R"]))
    _K_zeta = np.zeros_like(_K_R)
    scotty_2d_data["K_X_values"] = _K_R*_cos - _K_zeta/_R*_sin
    scotty_2d_data["K_Y_values"] = _K_R*_sin + _K_zeta/_R*_cos
    scotty_2d_data["K_Z_values"] = np.array(list(h5file["solver_output"]["K_Z"]))

    # Obtaining individual elements of all Psi_3D_Cartesian
    # scotty_2d_data["Psi_3D_cylindrical"] = list(h5file["solver_output"]["Psi_3D"][75:]) # REMEMBER to un-comment find_Psi_3D_lab_Cartesian below
    _Psi = np.array(h5file["analysis"]["Psi_3D_Cartesian"])
    scotty_2d_data["Psi_3D_cartesian_XX_Re"] = np.real(_Psi[:,0,0])
    scotty_2d_data["Psi_3D_cartesian_XY_Re"] = np.real(_Psi[:,0,1])
    scotty_2d_data["Psi_3D_cartesian_XZ_Re"] = np.real(_Psi[:,0,2])
    scotty_2d_data["Psi_3D_cartesian_YY_Re"] = np.real(_Psi[:,1,1])
    scotty_2d_data["Psi_3D_cartesian_YZ_Re"] = np.real(_Psi[:,1,2])
    scotty_2d_data["Psi_3D_cartesian_ZZ_Re"] = np.real(_Psi[:,2,2])
    scotty_2d_data["Psi_3D_cartesian_XX_Im"] = np.imag(_Psi[:,0,0])
    scotty_2d_data["Psi_3D_cartesian_XY_Im"] = np.imag(_Psi[:,0,1])
    scotty_2d_data["Psi_3D_cartesian_XZ_Im"] = np.imag(_Psi[:,0,2])
    scotty_2d_data["Psi_3D_cartesian_YY_Im"] = np.imag(_Psi[:,1,1])
    scotty_2d_data["Psi_3D_cartesian_YZ_Im"] = np.imag(_Psi[:,1,2])
    scotty_2d_data["Psi_3D_cartesian_ZZ_Im"] = np.imag(_Psi[:,2,2])

    # Converting first derivatives from cylindrical to cartesian
    _dH_dR    = np.array((h5file["analysis"]["dH_dR"]))
    _dH_dzeta = np.zeros_like(_dH_dR)
    scotty_2d_data["dH_dX"] = _dH_dR*np.cos(_zeta) - 1/_R*np.sin(_zeta)*_dH_dzeta
    scotty_2d_data["dH_dY"] = _dH_dR*np.sin(_zeta) + 1/_R*np.cos(_zeta)*_dH_dzeta
    scotty_2d_data["dH_dZ"] = np.array(h5file["analysis"]["dH_dZ"])
    
    _dH_dKR    = np.array((h5file["analysis"]["dH_dKR"]))
    _dH_dKzeta = np.array(_dH_dKR)
    scotty_2d_data["dH_dKx"] = _dH_dKR*np.cos(_zeta) - _dH_dKzeta*np.sin(_zeta)
    scotty_2d_data["dH_dKy"] = _dH_dKR*np.sin(_zeta) + _dH_dKzeta*np.cos(_zeta)
    scotty_2d_data["dH_dKz"] = np.array((h5file["analysis"]["dH_dKZ"]))

    # Getting second derivatives (cylindrical) from grad_grad_H, gradK_grad_H, gradK_gradK_H
    scotty_2d_data["grad_grad_H"] = np.array(h5file["analysis"]["grad_grad_H"])
    _d2H_dR2      = scotty_2d_data["grad_grad_H"][:,0,0] # top left each grad_grad_H
    _d2H_dR_dzeta = np.zeros_like(_d2H_dR2)
    _d2H_dR_dZ    = scotty_2d_data["grad_grad_H"][:,2,2] # top right of each grad_grad_H
    _d2H_dzeta2   = np.zeros_like(_d2H_dR2)
    _d2H_dzeta_dZ = np.zeros_like(_d2H_dR2)
    _d2H_dZ2      = scotty_2d_data["grad_grad_H"][:,2,2] # bottom right of each grad_grad_H

    scotty_2d_data["gradK_grad_H"] = np.array(h5file["analysis"]["gradK_grad_H"])
    _d2H_dKR_dR       = scotty_2d_data["gradK_grad_H"][:,0,0] # top left of each gradK_grad_H
    _d2H_dKR_dzeta    = np.zeros_like(_d2H_dKR_dR)
    _d2H_dKR_dZ       = scotty_2d_data["gradK_grad_H"][:,0,2] # top right of each gradK_grad_H
    _d2H_dKzeta_dR    = scotty_2d_data["gradK_grad_H"][:,1,0] # middle left of each gradK_grad_H
    _d2H_dKzeta_dzeta = np.zeros_like(_d2H_dKR_dR)
    _d2H_dKzeta_dZ    = scotty_2d_data["gradK_grad_H"][:,1,2] # middle right of each gradK_grad_H
    _d2H_dKZ_dR       = scotty_2d_data["gradK_grad_H"][:,2,0] # bottom left of each gradK_grad_H
    _d2H_dKZ_dzeta    = np.zeros_like(_d2H_dKR_dR)
    _d2H_dKZ_dZ       = scotty_2d_data["gradK_grad_H"][:,2,2] # bottom right of each gradK_grad_H

    scotty_2d_data["gradK_gradK_H"] = np.array(h5file["analysis"]["gradK_gradK_H"])
    _d2H_dKR2       = scotty_2d_data["gradK_gradK_H"][:,0,0] # top left of each gradK_gradK_H
    _d2H_dKR_dKzeta = scotty_2d_data["gradK_gradK_H"][:,0,1] # top middle of each gradK_gradK_H
    _d2H_dKR_dKZ    = scotty_2d_data["gradK_gradK_H"][:,0,2] # top right of each gradK_gradK_H
    _d2H_dKzeta2    = scotty_2d_data["gradK_gradK_H"][:,1,1] # middle middle of each gradK_gradK_H
    _d2H_dKzeta_dKZ = scotty_2d_data["gradK_gradK_H"][:,1,2] # middle right of each gradK_gradK_H
    _d2H_dKZ2       = scotty_2d_data["gradK_gradK_H"][:,2,2] # bottom right of each gradK_gradK_H

    # Converting second derivatives from cylindrical to cartesian
    scotty_2d_data["d2H_dX2"]   = _cos2*_d2H_dR2 - 2*_sincos/_R*_d2H_dR_dzeta + _sin2/_R2*_d2H_dzeta2 + _sin2/_R*_dH_dR + 2*_sincos/_R2*_dH_dzeta
    scotty_2d_data["d2H_dX_dY"] = _sincos*_d2H_dR2 + (_cos2-_sin2)/_R*_d2H_dR_dzeta - _sincos/_R2*_d2H_dzeta2 - _sincos/_R*_dH_dR + (_sin2-_cos2)/_R2*_dH_dzeta
    scotty_2d_data["d2H_dX_dZ"] = _cos*_d2H_dR_dZ - _sin/_R*_d2H_dzeta_dZ
    scotty_2d_data["d2H_dY2"]   = _sin2*_d2H_dR2 + 2*_sincos/_R*_d2H_dR_dzeta + _cos2/_R2*_d2H_dzeta2 + _cos2/_R*_dH_dR - 2*_sincos/_R2*_dH_dzeta
    scotty_2d_data["d2H_dY_dZ"] = _sin*_d2H_dR_dZ + _cos/_R*_d2H_dzeta_dZ
    scotty_2d_data["d2H_dZ2"]   = _d2H_dZ2

    scotty_2d_data["d2H_dX_dKx"] = _sin2/_R2*_d2H_dzeta2 + _cos2*_d2H_dKR_dR - _sincos/_R*_d2H_dKR_dzeta - _sincos*_d2H_dKzeta_dR + _sin2/_R*_dH_dKR + _sincos/_R2*_dH_dzeta
    scotty_2d_data["d2H_dX_dKy"] = _sin2*_d2H_dKR_dR + _sincos/_R*_d2H_dKR_dzeta + _R*_sincos*_d2H_dKzeta_dR + _cos2*_d2H_dKzeta_dzeta + _cos2/_R*_dH_dKR
    scotty_2d_data["d2H_dX_dKz"] = _cos*_d2H_dKZ_dR - _sin/_R*_d2H_dKZ_dzeta
    scotty_2d_data["d2H_dY_dKx"] = _sincos*_d2H_dKR_dR + _cos2/_R*_d2H_dKR_dzeta - _R*_sin2*_d2H_dKzeta_dR - _sincos*_d2H_dKzeta_dzeta - _sincos/_R*_dH_dKR - _dH_dKzeta
    scotty_2d_data["d2H_dY_dKy"] = _sin2*_d2H_dKR_dR + _sincos/_R*_d2H_dKR_dzeta + _R*_sincos*_d2H_dKzeta_dR + _cos2*_d2H_dKzeta_dzeta + _cos2/_R*_dH_dKR
    scotty_2d_data["d2H_dY_dKz"] = _sin*_d2H_dKZ_dR + _cos/_R*_d2H_dKZ_dzeta
    scotty_2d_data["d2H_dZ_dKx"] = _cos*_d2H_dKR_dZ - _R*_sin*_d2H_dKzeta_dZ
    scotty_2d_data["d2H_dZ_dKy"] = _sin*_d2H_dKR_dZ + _R*_cos*_d2H_dKzeta_dZ
    scotty_2d_data["d2H_dZ_dKz"] = _d2H_dKZ_dZ

    scotty_2d_data["d2H_dKx2"]    = _cos2*_d2H_dKR2 - 2*_R*_sincos*_d2H_dKR_dKzeta + _R2*_sin*_d2H_dKzeta2 + _R*_sin2*_dH_dKR + 2*_R2*_sincos*_dH_dKzeta
    scotty_2d_data["d2H_dKx_dKy"] = _sincos*_d2H_dKR2 + _R*(_cos2-_sin2)*_d2H_dKR_dKzeta - _R2*_sincos*_d2H_dKzeta2 - _R*_sincos*_dH_dKR + _R2*(_sin2-_cos2)*_dH_dKzeta
    scotty_2d_data["d2H_dKx_dKz"] = _cos*_d2H_dKR_dKZ - _R*_sin*_d2H_dKzeta_dKZ
    scotty_2d_data["d2H_dKy2"]    = _sin2*_d2H_dKR2 + 2*_R*_sincos*_d2H_dKR_dKzeta + _R2*_cos2*_d2H_dKzeta2 + _R*_cos2*_dH_dKR - 2*_R2*_sincos*_dH_dKzeta
    scotty_2d_data["d2H_dKy_dKz"] = _sin*_d2H_dKR_dKZ + _R*_cos*_d2H_dKzeta_dKZ
    scotty_2d_data["d2H_dKz2"]    = _d2H_dKZ2

# Because the derivatives are a bit ugly at the start, we actually begin 3d Scotty from some arbitrary
# tau value (I looked at the derivative graphs in the Google Slide deck and saw that the derivatives)
# become nice some time before the 75th tau value, so we choose this value to start plotting
_start_tau_index = 75
print(f"Value of tau at the {_start_tau_index}th index:", scotty_2d_data["tau_values"][_start_tau_index])
for i, key in enumerate(scotty_2d_data.keys()):
    scotty_2d_data[key] = scotty_2d_data[key][_start_tau_index:]





# Plotting routine
import matplotlib.pyplot as plt
import os

_pol = int(kwargs_dict["poloidal_launch_angle_Torbeam"])
_tor = int(kwargs_dict["poloidal_launch_angle_Torbeam"])
_path_directory = r"C:\Users\eduar\Desktop\to put on gslides\tokamak"
_path_folder_name = f"pol {_pol}, tor {_tor}"
_path = os.path.join(_path_directory, _path_folder_name)
if not os.path.exists(_path): os.makedirs(_path)

""" RAY TRAJECTORY """
# plot both together on a 3d plot
fig = plt.figure()
ax = plt.axes(projection = '3d')
ax.scatter(scotty_3d_data["X_coords"][0], scotty_3d_data["Y_coords"][0], scotty_3d_data["Z_coords"][0], color='black', label="Start Point")
ax.plot(scotty_3d_data["X_coords"], scotty_3d_data["Y_coords"], scotty_3d_data["Z_coords"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty Trajectory") # numerical
ax.plot(scotty_2d_data["X_coords"], scotty_2d_data["Y_coords"], scotty_2d_data["Z_coords"], color='red', linestyle='-', linewidth=2, zorder=1, label="2d Scotty Trajectory") # numerical
ax.set_xlabel("X-axis")
ax.set_ylabel("Y-axis")
ax.set_zlabel("Z-axis")
ax.set_title(f"Ray Trajectory, pol {_pol}, tor {_tor}")
ax.legend()
#ax.set_ylim(-0.1,0.1)
plt.show()
_path_fig_name = f"1. Ray Trajectory, pol {_pol}, tor {_tor}"
_path_full = os.path.join(_path, _path_fig_name)
plt.savefig(_path_full)
plt.close()

# plot both together on a 2d plot
plt.scatter(scotty_3d_data["X_coords"][0], scotty_3d_data["Z_coords"][0], color='black', label="Start Point")
plt.plot(scotty_3d_data["X_coords"], scotty_3d_data["Z_coords"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty Trajectory") # numerical
plt.plot(scotty_2d_data["X_coords"], scotty_2d_data["Z_coords"], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty Trajectory") # numerical
plt.xlabel("X-axis")
plt.ylabel("Z-axis")
plt.title(f"Ray Trajectory, pol {_pol}, tor {_tor}")
plt.legend()
plt.show()
_path_fig_name = f"2. Ray Trajectory, pol {_pol}, tor {_tor}"
_path_full = os.path.join(_path, _path_fig_name)
plt.savefig(_path_full)
plt.close()



""" POSITION """
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
axs = axs.flatten()
plt.tight_layout()
axs[0].plot(scotty_3d_data["tau_values"], scotty_3d_data["X_coords"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty X coord") # numerical
axs[0].plot(scotty_2d_data["tau_values"], scotty_2d_data["X_coords"], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty X coord") # numerical
axs[0].set_title(f"X coord vs tau, pol {_pol}, tor {_tor}")
axs[0].legend(loc="best")
axs[0].grid(True, which="both")
axs[1].plot(scotty_3d_data["tau_values"], scotty_3d_data["Y_coords"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty Y coord") # numerical
axs[1].plot(scotty_2d_data["tau_values"], scotty_2d_data["Y_coords"], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty Y coord") # numerical
axs[1].set_title(f"Y coord vs tau, pol {_pol}, tor {_tor}")
axs[1].legend(loc="best")
axs[1].grid(True, which="both")
axs[2].plot(scotty_3d_data["tau_values"], scotty_3d_data["Z_coords"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty Z coord") # numerical
axs[2].plot(scotty_2d_data["tau_values"], scotty_2d_data["Z_coords"], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty Z coord") # numerical
axs[2].set_title(f"Z coord vs tau, pol {_pol}, tor {_tor}")
axs[2].legend(loc="best")
axs[2].grid(True, which="both")
plt.show()
_path_fig_name = f"3. X, Y, Z coord vs tau, pol {_pol}, tor {_tor}"
_path_full = os.path.join(_path, _path_fig_name)
plt.savefig(_path_full)
plt.close()



""" WAVEVECTORS """
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
axs = axs.flatten()
plt.tight_layout()
axs[0].plot(scotty_3d_data["tau_values"], scotty_3d_data["K_X_values"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty K_X value") # numerical
axs[0].plot(scotty_2d_data["tau_values"], scotty_2d_data["K_X_values"], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty K_X value") # numerical
axs[0].set_title(f"K_X vs tau, pol {_pol}, tor {_tor}")
axs[0].legend(loc="best")
axs[0].grid(True, which="both")
axs[1].plot(scotty_3d_data["tau_values"], scotty_3d_data["K_Y_values"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty K_Y value") # numerical
axs[1].plot(scotty_2d_data["tau_values"], scotty_2d_data["K_Y_values"], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty K_Y value") # numerical
axs[1].set_title(f"K_Y vs tau, pol {_pol}, tor {_tor}")
axs[1].legend(loc="best")
axs[1].grid(True, which="both")
axs[2].plot(scotty_3d_data["tau_values"], scotty_3d_data["K_Z_values"], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty K_Z value") # numerical
axs[2].plot(scotty_2d_data["tau_values"], scotty_2d_data["K_Z_values"], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty K_Z value") # numerical
axs[2].set_title(f"K_Z vs tau, pol {_pol}, tor {_tor}")
axs[2].legend(loc="best")
axs[2].grid(True, which="both")
plt.show()
_path_fig_name = f"4. Kx, Ky, Kz vs tau, pol {_pol}, tor {_tor}"
_path_full = os.path.join(_path, _path_fig_name)
plt.savefig(_path_full)
plt.close()



""" PSI ELEMENTS """
fig, axs = plt.subplots(4, 3, figsize=(20, 15))
axs = axs.flatten()
plt.tight_layout()
counter_Re = 0
counter_Im = 3
for element, element_key in [("Psi_XX", "Psi_3D_cartesian_XX"), ("Psi_XY", "Psi_3D_cartesian_XY"), ("Psi_XZ", "Psi_3D_cartesian_XZ"), ("Psi_YY", "Psi_3D_cartesian_YY"), ("Psi_YZ", "Psi_3D_cartesian_YZ"), ("Psi_ZZ", "Psi_3D_cartesian_ZZ")]:
    if counter_Re == 3: counter_Re, counter_Im = 6, 9
    axs[counter_Re].plot(scotty_3d_data["tau_values"], scotty_3d_data[f"{element_key}_Re"], color='blue', linestyle='-', linewidth=2, zorder=1, label=f"3d Scotty {element}_Re value") # numerical
    axs[counter_Re].plot(scotty_2d_data["tau_values"], scotty_2d_data[f"{element_key}_Re"], color='red', linestyle='--', linewidth=2, zorder=1, label=f"2d Scotty {element}_Re value") # numerical
    axs[counter_Re].set_title(f"Re[{element}] vs tau, pol {_pol}, tor {_tor}")
    axs[counter_Re].grid(True, which="both")
    axs[counter_Re].legend(loc="best")
    counter_Re += 1
    axs[counter_Im].plot(scotty_3d_data["tau_values"], scotty_3d_data[f"{element_key}_Im"], color='blue', linestyle='-', linewidth=2, zorder=1, label=f"3d Scotty {element}_Im value") # numerical
    axs[counter_Im].plot(scotty_2d_data["tau_values"], scotty_2d_data[f"{element_key}_Im"], color='red', linestyle='--', linewidth=2, zorder=1, label=f"2d Scotty {element}_Im value") # numerical
    axs[counter_Im].set_title(f"Im[{element}] vs tau, pol {_pol}, tor {_tor}")
    axs[counter_Im].grid(True, which="both")
    axs[counter_Im].legend(loc="best")
    counter_Im += 1
plt.show()
_path_fig_name = f"5. Psi components vs tau, pol {_pol}, tor {_tor}"
_path_full = os.path.join(_path, _path_fig_name)
plt.savefig(_path_full)
plt.close()



""" FIRST DERIVATIVES """
fig, axs = plt.subplots(2, 3, figsize=(20, 10))
axs = axs.flatten()
plt.tight_layout()
counter = 0
for key in ["dH_dX", "dH_dY", "dH_dZ", "dH_dKx", "dH_dKy", "dH_dKz"]:
    axs[counter].plot(scotty_3d_data["tau_values"], scotty_3d_data[key], color='blue', linestyle='-', linewidth=2, zorder=1, label=f"3d Scotty")
    axs[counter].plot(scotty_2d_data["tau_values"], scotty_2d_data[key], color='red', linestyle='--', linewidth=2, zorder=1, label=f"2d Scotty")
    axs[counter].set_title(f"{key} vs tau, pol {_pol}, tor {_tor}")
    axs[counter].grid(True, which="both")
    axs[counter].legend(loc="best")
    counter += 1
plt.show()
_path_fig_name = f"6. 1st derivatives vs tau, pol {_pol}, tor {_tor}"
_path_full = os.path.join(_path, _path_fig_name)
plt.savefig(_path_full)
plt.close()



""" SECOND DERIVATIVES """
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
fig, axs = plt.subplots(7, 3, figsize=(15, 15))
axs = axs.flatten()
plt.tight_layout()
counter = 0
for key in ["d2H_dX2",     "d2H_dY2",     "d2H_dZ2",
            "d2H_dX_dY",   "d2H_dX_dZ",   "d2H_dY_dZ",
            "d2H_dKx2",    "d2H_dKy2",    "d2H_dKz2",
            "d2H_dKx_dKy", "d2H_dKx_dKz", "d2H_dKy_dKz",
            "d2H_dX_dKx",  "d2H_dX_dKy",  "d2H_dX_dKz",
            "d2H_dY_dKx",  "d2H_dY_dKy",  "d2H_dY_dKz",
            "d2H_dZ_dKx",  "d2H_dZ_dKy",  "d2H_dZ_dKz"]:
        # xmin, xmax = 0, 1002 # 0, 900
        # ymin, ymax = min(value[xmin:xmax+1]), max(value[xmin:xmax+1])
        # axs[counter].set_xlim(xmin, xmax)
        # axs[counter].set_ylim(ymin, ymax)
        axs[counter].plot(scotty_3d_data["tau_values"], scotty_3d_data[key], color='blue', linestyle='-', linewidth=2, zorder=1, label="3d Scotty")
        axs[counter].plot(scotty_2d_data["tau_values"], scotty_2d_data[key], color='red', linestyle='--', linewidth=2, zorder=1, label="2d Scotty")
        axs[counter].set_title(f"{key} vs tau, pol {_pol}, tor {_tor}")
        axs[counter].grid(True, which="both", axis="x")
        axs[counter].legend(loc="best")
        # if key == "d2H_dX2": axs[counter].set_ylim(-10,40)
        # if key == "d2H_dZ2": axs[counter].set_ylim(-10,0)
        # if key == "d2H_dX_dZ": axs[counter].set_ylim(-5,5)
        counter += 1
plt.show()
_path_fig_name = f"7. 2nd derivatives vs tau, pol {_pol}, tor {_tor}"
_path_full = os.path.join(_path, _path_fig_name)
plt.savefig(_path_full)
plt.close()

In [ ]:
# To find the data from the .h5 file generated by 2d Scotty

import h5py
import numpy as np
import pathlib

# Data location for B field and ne/polflux
h5file_path = r"C:\Users\eduar\OneDrive\OneDrive - EWIKARTA001\OneDrive - Nanyang Technological University\University\2. To Be Uploaded\Year 3, PH4416 Professional Attachment\Scotty in 3d Cartesian\tokamak benchmark\scotty_output.h5"

# h5 contains
# analysis, inputs, solver_output
with h5py.File(h5file_path, "r") as h5file:
    # To find the keys
    #print("Keys: %s" % h5file.keys())
    #print(h5file["solver_output"].keys())

    # To find starting conditions
    # print(h5file["solver_output"]["q_R"][0])
    # print(h5file["solver_output"]["q_zeta"][0])
    # print(h5file["solver_output"]["q_Z"][0])
    # print(h5file["solver_output"]["Psi_3D"][0])

    # the derivatives are not well-behaved at the start and ends
    # so let me just find the ray and beam parameters when tau is like 50-ish
    def find_tau_index(wanted_tau):
        tau_list = list(h5file["solver_output"]["tau"])
        length = len(tau_list)

        def binary_index_search(array, target):
            left, right = 0, len(array) - 1
            closest_index = -1

            while left <= right:
                mid = left + (right - left) // 2
                
                if array[mid] >= target:
                    closest_index = mid
                    right = mid - 1
                else:
                    left = mid + 1

            return closest_index
        
        wanted_tau_index = binary_index_search(tau_list, wanted_tau)
        return wanted_tau_index

    # now that I have the tau and tau index of where I should get
    # the data from in the 2d Scotty h5 file, let me find the starting
    # ray position, wavevector, and beam parameters
    from scotty.fun_general import find_q_lab_Cartesian, find_K_lab_Cartesian, find_Psi_3D_lab_Cartesian
    tau_adjustment = h5file["solver_output"]["tau"][75]

    q_R_temp = h5file["solver_output"]["q_R"][75] # [find_tau_index(start_tau)]
    q_zeta_temp = h5file["solver_output"]["q_zeta"][75] # [find_tau_index(start_tau)]
    q_Z_temp = h5file["solver_output"]["q_Z"][75] # [find_tau_index(start_tau)]
    q_X_temp, q_Y_temp, q_Z_temp = find_q_lab_Cartesian([q_R_temp, q_zeta_temp, q_Z_temp])

    K_R_temp = h5file["solver_output"]["K_R"][75] # [find_tau_index(start_tau)]
    K_zeta_temp = 0
    K_Z_temp = h5file["solver_output"]["K_Z"][75] # [find_tau_index(start_tau)]
    K_X_temp, K_Y_temp, K_Z_temp = find_K_lab_Cartesian([K_R_temp, K_zeta_temp, K_Z_temp], [q_R_temp, q_zeta_temp, q_Z_temp])
    
    Psi_3D_cylindrical = h5file["solver_output"]["Psi_3D"][75] # [find_tau_index(start_tau)]
    Psi_3D_cartesian = find_Psi_3D_lab_Cartesian(Psi_3D_cylindrical, q_R_temp, q_zeta_temp, K_R_temp, K_Z_temp)



    e_11 = h5file["analysis"]["epsilon_perp"][75]
    e_12 = h5file["analysis"]["epsilon_g"][75]
    e_bb = h5file["analysis"]["epsilon_para"][75]
    theta_m = h5file["analysis"]["theta_m"][75]

    sin_sq_theta_m = np.sin(theta_m)**2
    alpha = e_bb*sin_sq_theta_m + e_11*(1-sin_sq_theta_m)
    beta  = -e_11*e_bb*(1+sin_sq_theta_m) - (e_11**2-e_12**2)*(1-sin_sq_theta_m)
    gamma = e_bb*(e_11**2-e_12**2)

    print("Initial conditions")
    print()
    print("tau adjustment:", tau_adjustment)
    print()
    print("position:", [q_X_temp, q_Y_temp, q_Z_temp])
    print()
    print("wavevector:", [K_X_temp, K_Y_temp, K_Z_temp])
    print()
    print("Psi:", Psi_3D_cartesian)
    print()
    print()
    print()
    print("e_11:", e_11)
    print("e_12:", e_12)
    print("e_bb:", e_bb)
    print("sin_sq_theta_m:", sin_sq_theta_m)
    print("alpha:", alpha)
    print("beta:", beta)
    print("gamma:", gamma)